In [4]:
import csv
import datetime
import io
import json
import numpy as np
import pandas as pd
import requests

In [5]:
print(5)

5


In [3]:
year = 2019
month = '01'
link = f'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month}.csv'
taxi_data = requests.get(link)
df = pd.read_csv(io.StringIO(taxi_data.content.decode('utf-8')))

In [4]:
df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [40]:
def filter_df(df, year, month):
    df = df[df['tpep_pickup_datetime'].apply(lambda dt: int(dt[:4])) == year]
    df = df[df['tpep_dropoff_datetime'].apply(lambda dt: int(dt[:4])) == year]
    df = df[df['tpep_pickup_datetime'].apply(lambda dt: int(dt[5:7])) == month]
    return df[df['tpep_dropoff_datetime'].apply(lambda dt: int(dt[5:7])) == month]

def get_quantiles(series, n = 20):
    return dict(series.quantile([i / n for i in range(n + 1)]))

def get_distinct(series):
    return set(series)

def get_counts(series):
    return dict(series.value_counts())

def get_hours_of_pickup(df):
    return dict(df['tpep_pickup_datetime'].apply(lambda dt: int(dt[11:13])).value_counts())

In [11]:
df = filter_df(df, year, int(month))
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
7,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,N,163,229,1,6.5,0.5,0.5,1.25,0.0,0.3,9.05,NaN
8,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,N,229,7,1,13.5,0.5,0.5,3.70,0.0,0.3,18.50,NaN
9,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,N,141,234,1,10.0,0.5,0.5,1.70,0.0,0.3,13.00,NaN


In [35]:
def gather_info(df):
    info = {}
    for column in ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']:
        info[column] = get_quantiles(df[column])
    for column in ['VendorID', 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type']:
        info[column] = get_counts(df[column])
    info['hour_of_pickup'] = get_hours_of_pickup(df)
    info['num'] = len(df)
    info['total_paid'] = sum(df['total_amount'])
    info['total_tip'] = sum(df['tip_amount'])
    return info

In [36]:
gather_info(df)

{'passenger_count': 0.00    0.0
 0.05    1.0
 0.10    1.0
 0.15    1.0
 0.20    1.0
 0.25    1.0
 0.30    1.0
 0.35    1.0
 0.40    1.0
 0.45    1.0
 0.50    1.0
 0.55    1.0
 0.60    1.0
 0.65    1.0
 0.70    1.0
 0.75    2.0
 0.80    2.0
 0.85    2.0
 0.90    3.0
 0.95    5.0
 1.00    9.0
 Name: passenger_count, dtype: float64,
 'trip_distance': 0.00      0.00
 0.05      0.45
 0.10      0.60
 0.15      0.70
 0.20      0.80
 0.25      0.90
 0.30      1.01
 0.35      1.13
 0.40      1.26
 0.45      1.40
 0.50      1.53
 0.55      1.70
 0.60      1.90
 0.65      2.11
 0.70      2.40
 0.75      2.80
 0.80      3.40
 0.85      4.39
 0.90      6.40
 0.95     10.56
 1.00    831.80
 Name: trip_distance, dtype: float64,
 'fare_amount': 0.00      -362.00
 0.05         4.00
 0.10         5.00
 0.15         5.00
 0.20         5.50
 0.25         6.00
 0.30         6.50
 0.35         7.00
 0.40         7.50
 0.45         8.00
 0.50         8.50
 0.55         9.50
 0.60        10.00
 0.65        11

In [41]:
infos = {}
for year in ['2019', '2020']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        link = f'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month}.csv'
        taxi_data = requests.get(link)
        df = pd.read_csv(io.StringIO(taxi_data.content.decode('utf-8')))
        df = filter_df(df, int(year), int(month))
        infos[(year, month)] = gather_info(df)

/Users/alecmori_1/Github/jupyter_config/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
new_infos = {
    f'{year}-{month}': value
    for (year, month), value in infos.items()
}

In [52]:
import json

class NumpyEncoder(json.JSONEncoder):
     def default(self, obj):
         if isinstance(obj, np.int64):
             return int(obj)
         if isinstance(obj, np.float64):
             return float(obj)
         # Let the base class default method raise the TypeError
         return json.JSONEncoder.default(self, obj)



with open('taxi_summaries.json', 'w') as fh:
    json.dump(new_infos, fh, cls=NumpyEncoder)

NameError: name 'np' is not defined

In [6]:
with open('taxi_summaries.json') as fh:
    infos = json.load(fh)

In [7]:
with open('taxi+_zone_lookup.csv') as fh:
    csv_info = csv.reader(fh)
    burough_info = {}
    for i, row in enumerate(csv_info):
        if i == 0:
            continue
        burough_info[row[0]] = row[1]

In [8]:
rate_code_id = {
    1: 'Standard Rate',
    2: 'JFK',
    3: 'Newark',
    4: 'Nassau or Westchester',
    5: 'Negotiated Rate',
    6: 'Group Ride',
}
payment_type_id = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided Trip',
}

In [9]:
median_tips = [
    info['tip_amount'].get('0.5')
    for info in infos.values()
]
median_tips

[1.43,
 1.86,
 1.95,
 1.95,
 1.95,
 1.95,
 1.86,
 1.85,
 1.95,
 1.96,
 1.96,
 1.92,
 1.95,
 1.96,
 1.92,
 1.0,
 0.01,
 1.5,
 1.66,
 1.75,
 1.95,
 1.96,
 1.96,
 1.96]

In [10]:
{'DOLocationId',
 'PULocationId',
 'RatecodeID',
 'extra',
 'fare_amount',
 'hour_of_pickup',
 'improvement_surcharge',
 'mta_tax',
 'num',
 'passenger_count',
 'payment_type',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'total_paid',
 'total_tip',
 'trip_distance'}

{'DOLocationId',
 'PULocationId',
 'RatecodeID',
 'extra',
 'fare_amount',
 'hour_of_pickup',
 'improvement_surcharge',
 'mta_tax',
 'num',
 'passenger_count',
 'payment_type',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'total_paid',
 'total_tip',
 'trip_distance'}

In [11]:
import collections

table = []
headers = ['date', 'field_id', 'percentile', 'location', 'rate_code', 'payment_type', 'hour_of_pickup', 'value']
for date, info in infos.items():
    pickups = collections.defaultdict(int)
    dropoffs = collections.defaultdict(int)
    for field_id, info_2 in info.items():
        if isinstance(info_2, dict):
            for key, value in info_2.items():
                if 'PULocationID' == field_id:
                    pickups[burough_info[key]] += value
                elif 'DOLocationID' == field_id:
                    dropoffs[burough_info[key]] += value
                elif 'RatecodeID' == field_id:
                    table.append([date, field_id, None, None, rate_code_id.get(int(float(key)), 'Unknown'), None, None, value])
                elif 'payment_type' == field_id:
                    table.append([date, field_id, None, None, None, payment_type_id[int(float(key))], None, value])
                elif 'hour_of_pickup' == field_id:
                    table.append([date, field_id, None, None, None, None, key, value])
                elif field_id in {'VendorID'}:
                    continue
                else:
                    table.append([date, field_id, key, None, None, None, None, value])
        else:
            table.append([date, field_id, None, None, None, None, None, info_2])
    for pickup, count in pickups.items():
        table.append([date, 'PULocationId', None, pickup, None, None, None, count])
    for dropoff, count in dropoffs.items():
        table.append([date, 'DOLocationId', None, dropoff, None, None, None, count])

In [12]:
with open('taxi_processed.csv', 'w') as fh:
    writer = csv.writer(fh)
    writer.writerow(headers)
    for row in table:
        writer.writerow(row)